In [1]:
from api_keys import API_KEY
import requests, json

bills = requests.get(f'https://api.congress.gov/v3/bill/117/s?fromDateTime=2022-08-04T04:02:00Z&toDateTime=2022-09-30T04:03:00Z&sort=updateDate+asc&api_key={API_KEY}')

KeyboardInterrupt: 